In [1]:
from flask import Flask
from folium.plugins import HeatMap, HeatMapWithTime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import TimestampedGeoJson, TimeSliderChoropleth
from branca.element import Figure
from branca.colormap import linear
#import geopandas as gpd

ModuleNotFoundError: No module named 'geopandas'

In [2]:
app = Flask(__name__)
@app.route('/')
def index():
    ## FIRE DATA
    full_year = pd.read_csv('./modis-terra-day-full-year (1).csv', index_col = 0)
    full_year['brightness'] = full_year['brightness'] / full_year['brightness'].sum()
    # turning acquisition date to datetime so I can easily filter out certain dates
    full_year['acq_date'] = pd.to_datetime(full_year['acq_date'])
    # since we only have COVID data for our mapping software starting from 2/19/20 and up until 10/20/20 
    # then I will filter out those dates from the data I will use for the heat map
    full_year = full_year[(full_year['acq_date'] >= '2020-02-19') & (full_year['acq_date'] <= '2020-10-20')]
    full_year.groupby('acq_date').count().sort_values(by='brightness', ascending=False).head()
    # turning acquisition date back into a string so it's formatted correctly for the heat map
    full_year['acq_date'] = full_year['acq_date'].astype(str)
    # converted brightness to be on a scale of zero to one (format needed for HeatMapWithTime)
    # normalization equation from https://docs.tibco.com/pub/spotfire/7.0.0/doc/html/norm/norm_scale_between_0_and_1.htm
    full_year['brightness'] = (full_year['brightness'] - full_year['brightness'].min()) / (full_year['brightness'].max() - full_year['brightness'].min())
    # converted frp (fire radiative power) to be on a scale of zero to one (format needed for HeatMapWithTime)
    full_year['frp'] = (full_year['frp'] - full_year['frp'].min()) / (full_year['frp'].max() - full_year['frp'].min())
    # sorted acq_date
    full_year['acq_date'] = full_year['acq_date'].sort_values(ascending=True)
    # created a list that groups latitude, longitude and brightness (as the weight) by acquisition date
    data_bright = []
    for _, d in full_year.groupby('acq_date'):
        data_bright.append([[row['latitude'], row['longitude'], row['brightness']] for _, row in d.iterrows()])
    # created a time index for heatmap
    time_index = [k[0] for k in full_year.groupby('acq_date')]
    ## COVID DATA
    url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
    state_geo = f'{url}/us-states.json'
    state_data = pd.read_csv('./covidcast-indicator-combination-confirmed_incidence_num-2020-02-20-to-2020-10-22.csv', index_col=0)
    to_drop = state_data[state_data['geo_value'] == 'PR'].index
    state_data.drop(to_drop, inplace= True)
    state_dict = {state: number for state, number in zip(state_data['geo_value'], range(0, 52))}
    state_data['state_id'] = state_data['geo_value'].map(state_dict)
    state_data['geo_value'] = [i for i in state_data['geo_value']]
    state_data['value'].replace([-16, -46, -140, -8, -613, -1, -28, -31, -26, -398, -41], 0, inplace=True)
    bins = np.linspace(min(state_data['value']),max(state_data['value']), 11)
    state_data['color'] = pd.cut(state_data['value'], bins, labels = ['#e1f2f6','#b8d3e3','#b9d2e4','#a1c1db','#b8ccdf','#a4abd0','#9ba2cb','#8e6eb3','#927db8','#9b4395'], include_lowest = False)    
    state_data['color'].replace(to_replace = [np.nan], value = ['#EFF5F6'],inplace=True)
    state_data_color = state_data[['time_value', 'state_id', 'color']]
    state_data_color['state_id'] = state_data_color['state_id'].astype(str)
    state_data_color['time_value'] = pd.to_datetime(state_data_color['time_value'])
    state_data_color['time_value']=(state_data_color['time_value'].astype(int)// 10**9).astype('U10')
    covid_dict={}
    for i in state_data_color['state_id'].unique():
        covid_dict[i]={}
        for j in state_data_color[state_data_color['state_id']==i].set_index(['state_id']).values:   
            covid_dict[i][j[0]]={'color':j[1],'opacity':0.8}
    state_geojson=gpd.read_file(state_geo)
    state_dict = {state.upper(): number for state, number in zip(state_data['geo_value'], range(0, 52))}
    state_geojson['state_id']=state_geojson['id'].map(state_dict)
    state_geojson.drop(columns=['id', 'name'], inplace=True)
    start_coords = (39.833333, -98.133333)
    folium_map = folium.Map(width = 750, height = 500,left = '30%', location=start_coords, zoom_start=4)
    hm_bright = HeatMapWithTime(data_bright, index= time_index, name = 'Heat Map', auto_play=False, min_opacity=0.8, radius=5, overlay = True)
    TimeSliderChoropleth(
    state_geojson.set_index('state_id').to_json(),
    styledict=covid_dict, overlay = True,
    control = False,
    name = 'COVID TimeSlider'
    ).add_to(folium_map)
    folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_data,
    columns=['geo_value', 'value'],
    key_on='feature.id',
    fill_color='BuPu',
    fill_opacity=0.0,
    #line_opacity=0.2,
    legend_name='COVID-19 Incidence #',
    overlay = True
    ).add_to(folium_map)
    hm_bright.add_to(folium_map)
    folium.LayerControl().add_to(folium_map) 
    # creating a dataframe for June 14th to highlight it
    june14 = full_year[full_year['acq_date'] == '2020-06-14']
    def lat_long_map(latitude, longitude):
        # create a folium map
        fire_map = folium.Map(tiles='Stamen Terrain', width = 750, height = 500,left = '30%', location=start_coords, zoom_start=4)
        # add fire markers
        for lat, long in zip(latitude, longitude):
            folium.Marker(location=[lat, long], icon=folium.Icon(icon='fire', color='red'), tooltip=[lat, long]).add_to(fire_map)
        return fire_map
    m2 = lat_long_map(june14['latitude'], june14['longitude'])
    return f"""
   <html>
        <body>
            <h1> <center> Fires and COVID </center> </h1>
            <p1> <center> Interactive Time Slider </center> </p1>
            <p2> <center> blue markers represent fires and heat anomalies in the continental USA </center> </p2>
            <p3>{folium_map._repr_html_()}</p3>
            <p4>{m2._repr_html_()}</p4>
            <h2>  Thank you for visiting our app! </h2>
        </body>
    </html>
    """
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1

C:\Users\eboni\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
